In [2]:
import torch

import pandas as pd
from PIL import Image

from transformers import BitsAndBytesConfig
from transformers import AutoProcessor, LlavaForConditionalGeneration

In [3]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)

In [4]:
model_id = "llava-hf/llava-1.5-7b-hf"

In [5]:
processor = AutoProcessor.from_pretrained(model_id)
model = LlavaForConditionalGeneration.from_pretrained(model_id, quantization_config=quantization_config, device_map="auto")

processor_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/505 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.45k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/141 [00:00<?, ?B/s]

### Data

In [12]:
df = pd.read_csv("/Utilisateurs/umushtaq/emotion_analysis_comics/dataset_files/comics_pg_w_images.csv", index_col=0)

In [13]:
df_s = df.sample(10).reset_index(drop=True)

In [14]:
df_s

,file_name,page_nr,split,utterance,emotion_c,comics_title,comics_id,image_path
0,QC copy - 1521 - 40 Amor eterno 2.xlsx,24,TRAIN,"[""ROGER\u2026"", ""YES\u2026 YES\u2026 MY DARLIN...","[['sadness'], ['sadness'], ['anger', 'sadness'...",Love Everlasting #2,1521,/Utilisateurs/umushtaq/emotion_analysis_comics...
1,QC copy - 1910 - 35 Ladro_n de ladrones 1 - To...,10,TRAIN,"[""HERE. EVERYTHING I HAD SO FAR ON AUGUSTUS PA...","[['anger'], ['anger', 'sadness'], ['anger'], [...",Thief Of Thieves 1 - #11,1910,/Utilisateurs/umushtaq/emotion_analysis_comics...
2,QC copy - 1502 - 09 Mundos sin Liga de la Just...,4,TRAIN,"[""@LOBO THE CZARNIAN. BE WARNED."", ""@BY ORDER ...","[['anger'], ['anger'], ['anger'], ['surprise',...",Dark Crisis: Worlds Without a Justice League -...,1502,/Utilisateurs/umushtaq/emotion_analysis_comics...
3,QC copy - 1517 - 37 John Carpenter Historias p...,97,TRAIN,"[""NO WAY."", ""YOU MEAN\u2026"", ""YEP. SEE, UNLIK...","[['surprise'], ['fear', 'sadness'], ['sadness'...",John Carpenter's Tales for a Halloweenight #3,1517,/Utilisateurs/umushtaq/emotion_analysis_comics...
4,QC copy - 1517 - 37 John Carpenter Historias p...,71,TRAIN,"[""SO\u2026 HOW DO WE START?"", ""HOWEVER YOU WAN...","[['joy'], ['joy'], ['joy'], ['joy'], ['joy']]",John Carpenter's Tales for a Halloweenight #3,1517,/Utilisateurs/umushtaq/emotion_analysis_comics...
5,QC copy - 1560 - 36 Fantasmas vol. 1 - Ghosted...,9,TRAIN,"[""YOU'RE ALREADY HAUNTED, JACKSON. THAT'S WHY ...","[['anger', 'joy'], ['surprise', 'joy'], ['ange...",Fantasmas vol. 1 - Ghosted #3,1560,/Utilisateurs/umushtaq/emotion_analysis_comics...
6,QC copy - 2200 - Stillwater 13.xlsx,8,TEST,"[""I don't know what's happening."", ""I should b...","[['anger', 'fear'], ['anger', 'fear'], ['anger...",Stillwater #13,2200,/Utilisateurs/umushtaq/emotion_analysis_comics...
7,QC copy - 1508 - 48 Pequen_os Titanes 25.xlsx,19,TRAIN,"[""ANY BODY WANT SOME BUBBLE GUM?"", ""GUM!"", ""AA...","[['surprise', 'joy'], ['surprise', 'joy'], ['f...",Tiny Titans #25,1508,/Utilisateurs/umushtaq/emotion_analysis_comics...
8,QC copy - 1568 - 36 Fantasmas vol. 1 - Ghosted...,15,TRAIN,"[""Normally, I just send others to get their ha...","[['neutral'], ['neutral'], ['sadness'], ['disg...",Fantasmas vol. 1 - Ghosted #7,1568,/Utilisateurs/umushtaq/emotion_analysis_comics...
9,QC copy - 1560 - 36 Fantasmas vol. 1 - Ghosted...,12,TRAIN,"[""SPIRITUS ADEO MIHI! OSTENDO SUM VESTRI! DICO...","[['neutral'], ['neutral'], ['neutral'], ['disg...",Fantasmas vol. 1 - Ghosted #3,1560,/Utilisateurs/umushtaq/emotion_analysis_comics...


### Prompts

In [85]:
prompt = [
            "USER: <image>\nPlease describe this image, identifying the characters and the narrative.\nASSISTANT:",
]

In [86]:
images_l = df_s.image_path.tolist()

In [87]:
#prompts = prompt * len(images_l)

In [88]:
prompts = []
for _, row in df_s.iterrows():
    
    prompts.append(prompt[0].replace("<title>", str(row.comics_title)))

In [89]:
prompts

['USER: <image>\nPlease describe this image, identifying the characters and the narrative.\nASSISTANT:',
 'USER: <image>\nPlease describe this image, identifying the characters and the narrative.\nASSISTANT:',
 'USER: <image>\nPlease describe this image, identifying the characters and the narrative.\nASSISTANT:',
 'USER: <image>\nPlease describe this image, identifying the characters and the narrative.\nASSISTANT:',
 'USER: <image>\nPlease describe this image, identifying the characters and the narrative.\nASSISTANT:',
 'USER: <image>\nPlease describe this image, identifying the characters and the narrative.\nASSISTANT:',
 'USER: <image>\nPlease describe this image, identifying the characters and the narrative.\nASSISTANT:',
 'USER: <image>\nPlease describe this image, identifying the characters and the narrative.\nASSISTANT:',
 'USER: <image>\nPlease describe this image, identifying the characters and the narrative.\nASSISTANT:',
 'USER: <image>\nPlease describe this image, identifyin

In [90]:
images = [Image.open(image_path) for image_path in images_l]

In [91]:
inputs = processor(prompts, images, padding=True, return_tensors="pt").to("cuda")

In [92]:
for k,v in inputs.items():
  print(k,v.shape)

input_ids torch.Size([10, 602])
attention_mask torch.Size([10, 602])
pixel_values torch.Size([10, 3, 336, 336])


In [93]:
output = model.generate(**inputs, max_new_tokens=256)

In [94]:
generated_text = processor.batch_decode(output, skip_special_tokens=True)

for text in generated_text:
  print(text.split("ASSISTANT:")[-1])

 The image is a comic strip featuring a man and a woman lying next to each other. The woman has a lot of blood on her face, which suggests that she might be in a dangerous situation. The man is also covered in blood, indicating that they might be in a dangerous situation together.

The comic strip is divided into three panels, each showing a different moment in the story. The first panel shows the man and woman lying down, while the second panel shows the woman with blood on her face. The third panel shows the man also covered in blood, emphasizing the intensity of the situation.

The comic strip is written in English, and the characters' facial expressions and body language convey the emotions and tension of the scene.
 The image is a comic strip featuring a man and a woman. The man is wearing a suit and tie, and he appears to be engaged in a conversation with the woman. The woman is wearing glasses and is also dressed in a professional manner.

There are several chairs in the scene, 